In [1]:
import json
from itertools import chain
from math import ceil

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

/home/ubuntu/anaconda2/envs/fashion/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50
from keras import losses
from keras.layers import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [3]:
from keras import Model
from sklearn.model_selection import train_test_split

In [4]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

## Load ResNet model

In [5]:
base_model = ResNet50(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

In [14]:
# print(base_model.summary())

In [15]:
label_count=227

In [16]:
x = Flatten()(base_model.output)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(label_count, activation = 'sigmoid')(x)

In [17]:
head_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda2/envs/fashion/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [18]:
head_model.load_weights('results/best_weights.hdf5')

In [19]:
head_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
head_model.output_shape

(None, 227)

## Go through test images and predict

In [36]:
batch_size = 100
gen_test = ImageDataGenerator().flow_from_directory('images_test',
                                                    target_size=(224, 224),
                                                    class_mode=None, 
                                                    shuffle=False,
                                                    batch_size=batch_size)
num_images_test = len(gen_test.filenames)

Found 39706 images belonging to 1 classes.


In [37]:
img_ids_test_all = np.array([int(fn[5:-4]) for fn in gen_test.filenames])
img_ids_test_all.shape

(39706,)

### first 1k

In [56]:
test_chunks = []
test_id_chunks = []

In [57]:
num_images_test = 1000

start_batch = 0
batches = []
for i in range(ceil(num_images_test / gen_test.batch_size)):
    batches.append(gen_test.next())
    
img_arrays_test = np.array(list(chain(*batches)))
del batches
img_arrays_test = img_arrays_test[:num_images_test]

test_chunks.append(img_arrays_test)


In [58]:
i

9

In [60]:
test_id_chunks.append(
    img_ids_test_all[start_batch * batch_size: 
                     start_batch * batch_size + num_images_test]
)


In [61]:
test_id_chunks

[array([    1,    10,   100,  1000, 10000, 10001, 10002, 10003, 10004,
        10005, 10006, 10007, 10008, 10009,  1001, 10010, 10011, 10012,
        10013, 10014, 10015, 10016, 10017, 10018, 10019,  1002, 10020,
        10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029,
         1003, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037,
        10038, 10039,  1004, 10040, 10041, 10042, 10043, 10044, 10045,
        10046, 10047, 10048, 10049,  1005, 10050, 10051, 10052, 10053,
        10054, 10055, 10056, 10057, 10058, 10059,  1006, 10060, 10061,
        10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069,  1007,
        10070, 10071, 10072, 10073, 10074, 10075, 10076, 10077, 10078,
        10079,  1008, 10080, 10081, 10082, 10083, 10084, 10085, 10086,
        10087, 10088, 10089,  1009, 10090, 10091, 10092, 10093, 10094,
        10095, 10096, 10097, 10098, 10099,   101,  1010, 10100, 10101,
        10102, 10103, 10104, 10105, 10106, 10107, 10108, 10109,  1011,
      

In [63]:
prediction_chunks = []

In [65]:
%%time
prediction_chunks.append(head_model.predict(test_chunks[0]))

CPU times: user 20min 54s, sys: 1min 45s, total: 22min 39s
Wall time: 6min 11s


In [67]:
prediction_chunks[0].shape

(100, 227)

In [ ]:
num_images_test = 1000

start_batch = 1
batches = []
for i in range(ceil(num_images_test / gen_test.batch_size)):
    batches.append(gen_test.next())
    
img_arrays_test = np.array(list(chain(*batches)))
del batches
img_arrays_test = img_arrays_test[:num_images_test]

test_chunks.append(img_arrays_test)
